In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#setting
prompt_style = 'AutoCOT1'
data_count = 1
examle_count = 10


In [ ]:
# Configs
import config 

In [ ]:
# Run generator.js
import subprocess

params = ["200", "200"] # Enter the number of data. each prameter indicate the number of small, large map.

subprocess.run(["node", "generator.js"] + params)

In [ ]:
# Run placer.py
import placer

maps = placer.load_maps()
for map in maps:
    placer.assign_parameters(map, enemy_density=0.05, cohesion=0.3, treasure_density=0.01, range_multiplier=2, boss=True)
    placer.modify_map(map, group_min_dist=10, flag_try_count=50, enemy_sparsity=3)
placer.save_maps(maps)

In [ ]:
# Run labeler.py
import labeler

labeler.label(placed_path=config.PLACED_PATH, labelled_path=config.LABELLED_PATH, file_count=4, difficulty_curve_interval=5)

In [ ]:
# Generate preprocessed
from preprocessed_data_generator import preprocessed_data_generate

preprocessed_data_generate(data_count, examle_count, prompt_style)




In [ ]:
# Run comparator.py
import comparator

comparator.compare(preprocessed_path=config.PREPROCESSED_PATH, compared_path=config.COMPARED_PATH, file_count=1)

In [ ]:
# Calculate statistics
import statistician

diff_param_name_list = [
    "map_size",
    "room_count",
    "enemy_count",
    "treasure_count",
]

after_param_name_list = ["playability", "other_ASCII_count", "empty_validation"]

print(statistician.calc_abs_diff_mean_std_from_path(param_name_list=diff_param_name_list))
print(statistician.calc_after_mean_std_from_path(param_name_list=after_param_name_list))